In [ ]:
# We have a bunch of emails on disk

import glob
import io
import spacy
from collections import Counter
import pandas as pd
import email
import re

In [ ]:
eml_df = pd.read_csv('../../data/email-training.csv')

eml_df.columns

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report

# Normal distribution bell curve: Gaussian
# Beroulli: 

clf = BernoulliNB(alpha=0.0)

X = eml_df[eml_df.columns[~eml_df.columns.isin(['SPAM_VALUE'])]].fillna(value=0)
y = eml_df['SPAM_VALUE']

clf.fit(X, y)

# Why is the value what it is? What would you do to fix this?

print(classification_report(clf.predict(X), y))

In [ ]:
eml_df.groupby('SPAM_VALUE').size()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = BernoulliNB(alpha=0.06)
# clf = GaussianNB()

clf.fit(X_train, y_train)

print(classification_report(clf.predict(X_test), y_test))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(clf.predict(X_test), y_test)

I was able to get ~0.94 for precision on both Ham(1) and Spam(0). That's pretty good! The recall isn't so good actually. A 0.60 recall isn't so great, although we could probably do better on parsing the Emails themselves. Think of some reasons how this result could get better.

# BONUS word2vec

Word2Vec is an amazing model to fit natural language to a vectorized representation. Matter of fact it's called representation learning and basically will take text and output something that we can do math on!

Below I've done the heavy lifting of giving you a word2vec file to train on. Try out GaussianNB and see what happens. Is it any better or worse?

In [ ]:
eml_df = pd.read_csv('../../data/emails-word2vec.csv')

eml_df.head()

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

clf = LinearSVC(C=50)

X = eml_df[eml_df.columns[~eml_df.columns.isin(['SPAM_VALUE'])]].fillna(value=0)
y = eml_df['SPAM_VALUE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf.fit(X_train, y_train)

print(classification_report(clf.predict(X_test), y_test))